In [1]:
import os
import gc
import sys
import glob
import time
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import tensorflow as tf
import keras
import keras.backend as K
from keras import regularizers
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten, \
                                    Concatenate, PReLU, TimeDistributed, LSTM, Masking
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
#from keras_tqdm import TQDMNotebookCallback

sys.path.insert(0, "../../python")
from common import *
from t_notify import Notify
from DataLoader import DataLoader, read_hdf_lock

Using TensorFlow backend.


In [2]:
def compile_model(model, learning_rate):
    #opt = keras.optimizers.Adam(lr=learning_rate)
    opt = keras.optimizers.Nadam(lr=learning_rate, schedule_decay=1e-4)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = [
        "accuracy", TauLosses.tau_crossentropy, TauLosses.tau_crossentropy_v2,
        TauLosses.Le, TauLosses.Lmu, TauLosses.Ljet,
        TauLosses.He, TauLosses.Hmu, TauLosses.Htau, TauLosses.Hjet,
        TauLosses.Hcat_e, TauLosses.Hcat_mu, TauLosses.Hcat_jet, TauLosses.Hbin,
        TauLosses.Hcat_eInv, TauLosses.Hcat_muInv, TauLosses.Hcat_jetInv,
        TauLosses.Fe, TauLosses.Fmu, TauLosses.Fjet, TauLosses.Fcmb
    ]
    model.compile(loss=TauLosses.tau_crossentropy_v2, optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [3]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [4]:
class TimeCheckpoint(Callback):
    def __init__(self, time_interval, file_name_prefix):
        self.time_interval = time_interval
        self.file_name_prefix = file_name_prefix
        self.initial_time = time.time()
        self.last_check_time = self.initial_time
    
    def on_batch_end(self, batch, logs=None):
        if self.time_interval is None or batch % 100 != 0: return
        current_time = time.time()
        delta_t = current_time - self.last_check_time
        if delta_t >= self.time_interval:
            abs_delta_t_h = (current_time - self.initial_time) / 60. / 60.
            read_hdf_lock.acquire()
            self.model.save('{}_historic_b{}_{:.1f}h.h5'.format(self.file_name_prefix, batch, abs_delta_t_h))
            read_hdf_lock.release()
            self.last_check_time = current_time
    
    def on_epoch_end(self, epoch, logs=None):
        read_hdf_lock.acquire()
        self.model.save('{}_e{}.h5'.format(self.file_name_prefix, epoch))
        read_hdf_lock.release()
        Notify("Epoch {} is ended.".format(epoch))

In [5]:
def run_training(train_suffix, model_name, data_loader, epoch, n_epochs):

    train_name = '%s_%s' % (model_name, train_suffix)  
    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    time_checkpoint = TimeCheckpoint(12*60*60, train_name)
    callbacks = [time_checkpoint, csv_log]
    fit_hist = model.fit_generator(data_loader.generator(True), validation_data=data_loader.generator(False),
                                   steps_per_epoch=data_loader.steps_per_epoch, validation_steps=data_loader.validation_steps,
                                   callbacks=callbacks, epochs=n_epochs, initial_epoch=epoch, verbose=1)

    read_hdf_lock.acquire()
    model.save("%s_final.hdf5" % train_name)
    read_hdf_lock.release()
    return fit_hist

In [6]:
loader = DataLoader('N:/tau-ml/tuples-v2-t3/training/part_*.h5', netConf_full, 500, 10000, validation_size=6100000,
                    max_queue_size=40, n_passes=-1, return_grid=True)

print(loader.total_size, loader.data_size, loader.validation_size)

72491602 66391602 6100000


In [8]:
TauLosses.SetSFs(1, 2.5, 5, 1.5)
print(TauLosses.Le_sf, TauLosses.Lmu_sf, TauLosses.Ltau_sf, TauLosses.Ljet_sf)
model_name = "DeepTau2017v2p5"
model = LoadModel('DeepTau2017v2p5_step1_final.hdf5')
compile_model(model, 1e-3)

0.4 1.0 2.0 0.6000000000000001
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [9]:
fit_hist = run_training('step{}'.format(2), model_name, loader, 0, 5)

Epoch 1/5
132784/132784 [==============================] - 69979s 527ms/step - loss: 0.2815 - acc: 0.8599 - tau_crossentropy: 0.1033 - tau_crossentropy_v2: 0.2777 - Le: 0.0661 - Lmu: 0.0129 - Ljet: 0.1066 - He: 0.1965 - Hmu: 0.0344 - Htau: 0.0721 - Hjet: 0.1793 - Hcat_e: 0.0310 - Hcat_mu: 0.0011 - Hcat_jet: 0.0684 - Hbin: 0.0637 - Hcat_eInv: 0.1654 - Hcat_muInv: 0.0332 - Hcat_jetInv: 0.1110 - Fe: 0.0079 - Fmu: 0.0015 - Fjet: 0.0100 - Fcmb: 0.0402 - weighted_acc: 0.8391 - weighted_tau_crossentropy: 0.1098 - weighted_tau_crossentropy_v2: 0.2815 - weighted_Le: 0.0835 - weighted_Lmu: 0.0235 - weighted_Ljet: 0.0881 - weighted_He: 0.2004 - weighted_Hmu: 0.0587 - weighted_Htau: 0.0885 - weighted_Hjet: 0.1379 - weighted_Hcat_e: 0.0320 - weighted_Hcat_mu: 0.0012 - weighted_Hcat_jet: 0.0403 - weighted_Hbin: 0.0482 - weighted_Hcat_eInv: 0.1684 - weighted_Hcat_muInv: 0.0575 - weighted_Hcat_jetInv: 0.0976 - weighted_Fe: 0.0098 - weighted_Fmu: 0.0016 - weighted_Fjet: 0.0060 - weighted_Fcmb: 0.0296 -

KeyboardInterrupt: 

In [9]:
TauLosses.SetSFs(1, 2.5, 5, 1.5)
model_name = "DeepTau2017v2p5"
model = LoadModel('DeepTau2017v2p5_step2_e1.h5')
opt_weights = K.batch_get_value(model.optimizer.weights)
compile_model(model, 1e-3)
model._make_train_function()
model.optimizer.set_weights(opt_weights)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [10]:
fit_hist = run_training('step{}'.format(2), model_name, loader, 2, 5)

Epoch 3/5
132784/132784 [==============================] - 69903s 526ms/step - loss: 0.2941 - acc: 0.9307 - tau_crossentropy: 0.0853 - tau_crossentropy_v2: 0.2992 - Le: 0.0512 - Lmu: 0.0093 - Ljet: 0.0925 - He: 0.0693 - Hmu: 0.0400 - Htau: 0.0752 - Hjet: 0.0740 - Hcat_e: 0.0159 - Hcat_mu: 9.6190e-04 - Hcat_jet: 0.0290 - Hbin: 0.0527 - Hcat_eInv: 0.0534 - Hcat_muInv: 0.0390 - Hcat_jetInv: 0.0450 - Fe: 0.0121 - Fmu: 0.0013 - Fjet: 0.0181 - Fcmb: 0.0317 - weighted_acc: 0.9107 - weighted_tau_crossentropy: 0.0957 - weighted_tau_crossentropy_v2: 0.2941 - weighted_Le: 0.0706 - weighted_Lmu: 0.0184 - weighted_Ljet: 0.0818 - weighted_He: 0.0737 - weighted_Hmu: 0.0742 - weighted_Htau: 0.0913 - weighted_Hjet: 0.0547 - weighted_Hcat_e: 0.0170 - weighted_Hcat_mu: 9.1101e-04 - weighted_Hcat_jet: 0.0154 - weighted_Hbin: 0.0404 - weighted_Hcat_eInv: 0.0567 - weighted_Hcat_muInv: 0.0733 - weighted_Hcat_jetInv: 0.0393 - weighted_Fe: 0.0138 - weighted_Fmu: 0.0013 - weighted_Fjet: 0.0099 - weighted_Fcmb: 

ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [7]:
TauLosses.SetSFs(1, 2.5, 5, 1.5)
print(TauLosses.Le_sf, TauLosses.Lmu_sf, TauLosses.Ltau_sf, TauLosses.Ljet_sf)
model_name = "DeepTau2017v2p5"
model = LoadModel('DeepTau2017v2p5_step2_e2.h5')
opt_weights = K.batch_get_value(model.optimizer.weights)
compile_model(model, 1e-3)
model._make_train_function()
model.optimizer.set_weights(opt_weights)

0.4 1.0 2.0 0.6000000000000001
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [8]:
fit_hist = run_training('step{}'.format(2), model_name, loader, 3, 5)

Epoch 4/5
132784/132784 [==============================] - 70187s 529ms/step - loss: 0.2881 - acc: 0.9392 - tau_crossentropy: 0.0830 - tau_crossentropy_v2: 0.2943 - Le: 0.0493 - Lmu: 0.0085 - Ljet: 0.0913 - He: 0.0652 - Hmu: 0.0234 - Htau: 0.0738 - Hjet: 0.0698 - Hcat_e: 0.0154 - Hcat_mu: 9.4921e-04 - Hcat_jet: 0.0280 - Hbin: 0.0525 - Hcat_eInv: 0.0498 - Hcat_muInv: 0.0225 - Hcat_jetInv: 0.0419 - Fe: 0.0118 - Fmu: 0.0013 - Fjet: 0.0178 - Fcmb: 0.0316 - weighted_acc: 0.9215 - weighted_tau_crossentropy: 0.0926 - weighted_tau_crossentropy_v2: 0.2881 - weighted_Le: 0.0683 - weighted_Lmu: 0.0169 - weighted_Ljet: 0.0806 - weighted_He: 0.0675 - weighted_Hmu: 0.0545 - weighted_Htau: 0.0897 - weighted_Hjet: 0.0513 - weighted_Hcat_e: 0.0164 - weighted_Hcat_mu: 8.3065e-04 - weighted_Hcat_jet: 0.0149 - weighted_Hbin: 0.0404 - weighted_Hcat_eInv: 0.0511 - weighted_Hcat_muInv: 0.0536 - weighted_Hcat_jetInv: 0.0364 - weighted_Fe: 0.0133 - weighted_Fmu: 0.0011 - weighted_Fjet: 0.0097 - weighted_Fcmb: 

In [ ]:
fit_hist = run_training('step{}'.format(2), model_name, loader, 5, 6)

Epoch 6/6
132784/132784 [==============================] - 70327s 530ms/step - loss: 0.2832 - acc: 0.9407 - tau_crossentropy: 0.0809 - tau_crossentropy_v2: 0.2889 - Le: 0.0476 - Lmu: 0.0079 - Ljet: 0.0898 - He: 0.0599 - Hmu: 0.0191 - Htau: 0.0719 - Hjet: 0.0689 - Hcat_e: 0.0150 - Hcat_mu: 9.3813e-04 - Hcat_jet: 0.0275 - Hbin: 0.0521 - Hcat_eInv: 0.0449 - Hcat_muInv: 0.0181 - Hcat_jetInv: 0.0414 - Fe: 0.0116 - Fmu: 0.0012 - Fjet: 0.0176 - Fcmb: 0.0315 - weighted_acc: 0.9233 - weighted_tau_crossentropy: 0.0905 - weighted_tau_crossentropy_v2: 0.2832 - weighted_Le: 0.0665 - weighted_Lmu: 0.0164 - weighted_Ljet: 0.0792 - weighted_He: 0.0609 - weighted_Hmu: 0.0497 - weighted_Htau: 0.0880 - weighted_Hjet: 0.0509 - weighted_Hcat_e: 0.0160 - weighted_Hcat_mu: 8.7898e-04 - weighted_Hcat_jet: 0.0145 - weighted_Hbin: 0.0398 - weighted_Hcat_eInv: 0.0450 - weighted_Hcat_muInv: 0.0488 - weighted_Hcat_jetInv: 0.0364 - weighted_Fe: 0.0130 - weighted_Fmu: 0.0011 - weighted_Fjet: 0.0096 - weighted_Fcmb: 